In [7]:
from IPython.utils import io
from IPython import get_ipython
with io.capture_output() as captured:  
   get_ipython().run_line_magic('run', '1.Preprocessing.ipynb')

# Imports

In [8]:
import os
import librosa
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# Config

In [9]:
from config import run_config

run_config()

# Feature Extraction

In [10]:
# Extract MFCC features (or replace with other feature extractors)
df['features'] = df['audio_data'].progress_apply(lambda y: librosa.feature.mfcc(y=y, sr=16000, n_mfcc=40).mean(axis=1))

X = np.stack(df['features'].values)
y = df['label'].values

100%|██████████| 489/489 [00:03<00:00, 144.22it/s]


In [11]:
# Train/val/test split
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=0)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.3, random_state=0)